# Project 1

## Part 1: Basic Pathfinding
### Task choice 1: Depth-first Search

In [8]:
import os
import time
class Maze:
    def __init__(self, maze):
        # Initialize the Maze object with the given maze
        self.maze = maze
        self.start = None
        self.goal = None
        self.free_spaces = []
        self.walls = []
        self.visited = []
        self.path = []
        self.backtracked = []
        self.directions = {}
        
        # Store the maze data in a 2d-array where:
        #      'P': the starting point
        #      '.': the goal point
        #      ' ': free space that can be navigated through
        #      '%': walls that cannot be navigated through
        for i in range(len(maze)):
            for j in range(len(maze[i])):
                if maze[i][j] == 'P':
                    self.start = (i, j)
                elif maze[i][j] == '.':
                    self.goal = (i, j)
                elif maze[i][j] == ' ':
                    self.free_spaces.append((i, j))
                elif maze[i][j] == '%':
                    self.walls.append((i, j))

    # Get the neighboring positions of a given position
    def get_neighbors(self, position):
        i, j = position
        neighbors = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
        return [pos for pos in neighbors if self.maze[pos[0]][pos[1]] in ['P', '.', ' '] and pos not in self.visited]

    # Used for directional result output
    def get_direction(self, pos1, pos2):
        # Get the direction from pos1 to pos2
        if pos2[0] > pos1[0]:
            return "↓"
        elif pos2[0] < pos1[0]:
            return "↑"
        elif pos2[1] > pos1[1]:
            return "→"
        elif pos2[1] < pos1[1]:
            return "←"
        else:
            # If the positions are the same, return the direction of the last movement
            return self.path[-1][1] if len(self.path) > 1 else None

    # =====================================================
    # Depth First Search (recursive)
    # 
    # Helper functions: get_direction, get_neighbors
    #
    # Author: Jacob Thieret
    #
    # Description: Recursively visits each neighboring position of the current position 
    #              until the goal is found or all possible paths have been exhausted. 
    #              Backtracking is performed by removing the current position from the path if 
    #              none of its neighbors lead to the goal. The process repeats until the goal is
    #              found or all positions have been visited.
    #
    # ====================================================
    def DepthFirstSearch(self, position=None):
        if position is None:
            position = self.start
        # Adds the position to the visited list.
        self.visited.append(position)
        
        # Determines the direction of movement from the previous position to the current one and adds it to the directions dictionary.
        if self.path:
            direction = self.get_direction(self.path[-1], position)
            self.directions[position] = direction
        
        # Adds the current position to the path.
        self.path.append(position)
        
        # Checks if the current position is the goal. If yes, it outputs a success message and returns True.
        if position == self.goal:
            print("Goal found: " + str(position))
            return True
        
        # If the current position is not the goal, it gets the list of unvisited neighboring positions and recursively calls DepthFirstSearch on each one.
        neighbors = self.get_neighbors(position)
        for neighbor in neighbors:
            if self.DepthFirstSearch(neighbor):
                return True
            
        # If none of the neighbors lead to the goal (i.e., DepthFirstSearch returns False for all of them), 
        # it removes the current position from the path (backtracking) and adds it to the backtracked list, then returns False
        self.path.pop()
        self.backtracked.append(position)
        return False
    
    # execute dfs search, measures the time it takes to run
    def solve(self):
        start_time = time.time()
        self.DepthFirstSearch()
        end_time = time.time()
        execution_time = (end_time - start_time) * 1000
        print(f"Execution time: {round(execution_time, 8)} ms")
    
    # Rough calculation of the complexity of the maze by counting the number of vertices and edges
    def calculate_complexity(self):
        num_vertices = len(self.maze) * len(self.maze[0])  # total number of cells
        num_edges = sum(sum(1 for cell in row if cell != '%') for row in self.maze) * 4  # total number of connections
        print(f"Number of vertices: {num_vertices}")
        print(f"Number of edges: {num_edges}")
    
    # ===================================
    # Output Functions
    # ===================================

    # basic output
    # Writes the maze to a file, marking the visited positions with dots.
    def write_solution_dots(self, filename):
        solution_maze = [list(row) for row in self.maze]
        for i, j in self.visited:
            if solution_maze[i][j] == ' ':
                solution_maze[i][j] = '.'
        directory = os.path.dirname(filename)
        if directory:
            os.makedirs(directory, exist_ok=True)
        with open(filename, 'w') as file:
            for row in solution_maze:
                file.write(''.join(row) + '\n')

    # Directional Output [console]
    # Write the solved maze with directional arrows and bidirectional symbols to a file or to the console
    def write_solution(self, filename, toFile=False):
        print("\n")
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        if toFile is True:
            directory = os.path.dirname(filename)
            if directory:
                os.makedirs(directory, exist_ok=True)
            with open(filename, 'w', encoding='utf-8') as file:
                for row in solution_maze:
                    row = ['S' if cell == 'P' else cell for cell in row]
                    row = ['G' if cell == '.' else cell for cell in row]
                    row = ['▓' if cell == '%' else cell for cell in row]
                    file.write(''.join(row) + '\n')
        else:
            for row in solution_maze:
                row = ['S' if cell == 'P' else cell for cell in row]
                row = ['G' if cell == '.' else cell for cell in row]
                row = ['▓' if cell == '%' else cell for cell in row]
                print(''.join(row))

    # Print the number of steps taken and the path taken
    def print_path(self, b):
        # Number of steps taken
        print(f"Number of steps taken: {len(self.visited)}")
        print(f"Shortest path length found by DFS: {len(self.path)}")
        if b:
            print("Path taken:")
            for i in range(len(self.path)):
                # The start position
                if i == 0:
                    print(f"Started at {self.path[i]}")
                # Intermediate positions
                elif i < len(self.path) - 1:
                    direction = self.get_direction(self.path[i-1], self.path[i])
                    print(f"Moved {self.path[i-1]} {direction} {self.path[i]}")
                # The goal position
                else:
                    direction = self.get_direction(self.path[i-1], self.path[i])
                    print(f"Moved {direction} from {self.path[i-1]} to the goal at {self.path[i]}")          

# function to read in the maze
def read_maze_from_file(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze


In [9]:
smallMaze = read_maze_from_file('Maze/smallMaze.lay')

# Initialize small maze object and perform the DFS
smallMaze_obj = Maze(smallMaze)

smallMaze_obj.solve()
smallMaze_obj.calculate_complexity()
# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
smallMaze_obj.print_path(True)

# False to write solution to console, true to write to file
smallMaze_obj.write_solution('Maze/solutions/smallMaze-solution.lay', False)



Goal found: (8, 1)
Execution time: 0.0 ms
Number of vertices: 220
Number of edges: 376
Number of steps taken: 54
Shortest path length found by DFS: 30
Path taken:
Started at (3, 11)
Moved (3, 11) ← (3, 10)
Moved (3, 10) ← (3, 9)
Moved (3, 9) ← (3, 8)
Moved (3, 8) ← (3, 7)
Moved (3, 7) ← (3, 6)
Moved (3, 6) ↓ (4, 6)
Moved (4, 6) ↓ (5, 6)
Moved (5, 6) ↓ (6, 6)
Moved (6, 6) → (6, 7)
Moved (6, 7) → (6, 8)
Moved (6, 8) ↑ (5, 8)
Moved (5, 8) → (5, 9)
Moved (5, 9) → (5, 10)
Moved (5, 10) → (5, 11)
Moved (5, 11) → (5, 12)
Moved (5, 12) ↓ (6, 12)
Moved (6, 12) ↓ (7, 12)
Moved (7, 12) ← (7, 11)
Moved (7, 11) ← (7, 10)
Moved (7, 10) ↓ (8, 10)
Moved (8, 10) ← (8, 9)
Moved (8, 9) ← (8, 8)
Moved (8, 8) ← (8, 7)
Moved (8, 7) ← (8, 6)
Moved (8, 6) ← (8, 5)
Moved (8, 5) ← (8, 4)
Moved (8, 4) ← (8, 3)
Moved (8, 3) ← (8, 2)
Moved ← from (8, 2) to the goal at (8, 1)


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓↕▓▓↔↔↔↔↔↔↔↕▓ ▓      ▓
▓↔↔↔↕▓▓▓▓▓▓↕▓ ▓▓▓▓▓▓ ▓
▓▓▓▓▓▓←←←←←S  ▓      ▓
▓↕↔↔↔▓↓▓▓▓▓▓▓ ▓▓ ▓▓▓▓▓
▓↕▓▓▓▓↓▓↑→→→→    ▓   ▓
▓

In [10]:
mediumMaze = read_maze_from_file('Maze/mediumMaze.lay')

# Initialize small maze object and perform the DFS
mediumMaze_obj = Maze(mediumMaze)

mediumMaze_obj.solve()
mediumMaze_obj.calculate_complexity()
# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
mediumMaze_obj.print_path(False)

# Write to a solution directional file, and print directional to console
mediumMaze_obj.write_solution('Maze/solutions/mediumMaze-solution.lay', False)



Goal found: (16, 1)
Execution time: 2.00009346 ms
Number of vertices: 648
Number of edges: 1096
Number of steps taken: 259
Shortest path length found by DFS: 165


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓↕↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔S▓
▓↕▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓↕▓▓▓▓▓▓▓▓↓▓
▓↕▓▓↔↔↕▓↔↔↕▓←←←←←↑▓▓▓▓▓▓▓↕↔↔▓▓←←←←↓▓
▓↕▓▓↕▓↕▓↕▓↕▓↓▓▓▓▓↑▓▓▓▓▓▓▓▓▓↕▓▓↓▓▓▓▓▓
▓↕▓▓↕▓↕▓↕▓↕▓↓→→  ←←←↑↔↔↔↕▓▓↕▓▓↓→→→→▓
▓↕▓▓↕▓↕▓↕▓↕▓↕▓↓▓▓▓▓↕↑▓▓▓↔↔↔↕▓▓▓▓▓▓↓▓
▓↕▓↔↕▓↕▓↕▓↔↔↕▓↓→→→▓▓↑▓▓▓▓▓▓▓▓ ←←←←↓▓
▓↕▓▓↕▓↕▓↕▓▓▓▓▓▓▓▓↓▓▓←←←←←←←↑▓▓↓▓▓▓▓▓
▓↕▓▓↕▓↔↔↕▓▓←←←←←←↓▓▓▓▓▓▓▓▓▓↑▓▓↓→→→→▓
▓↔↔↔↕▓▓▓▓▓▓↓▓▓▓▓▓▓▓↑→→→→→▓▓↑▓▓▓▓▓▓↓▓
▓▓▓▓▓▓←←←←←↓▓↑→→→→→→▓▓▓▓↓▓▓↑▓ ←←←←↓▓
▓←←←←←↓▓▓▓▓▓▓↑▓▓▓▓▓↕▓←←←↓▓▓↑▓▓↓▓▓▓▓▓
▓↓▓▓▓▓▓▓↑→→→→→▓←←←←←←↓▓▓▓▓▓↑▓▓↓↑→→→▓
▓↓→→→→→→→▓▓▓▓▓▓↓▓▓▓▓▓▓▓▓▓▓▓↑▓▓↓→▓▓↓▓
▓▓▓▓▓▓▓▓▓▓←←←←←↓           ↑▓▓▓▓▓▓↓▓
▓G←←←←←←←←↓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓←←←←←←←↓▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓


In [11]:
bigMaze = read_maze_from_file('Maze/bigMaze.lay')

# Initialize small maze object and perform the DFS
bigMaze_obj = Maze(bigMaze)

bigMaze_obj.solve()
bigMaze_obj.calculate_complexity()
# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
bigMaze_obj.print_path(False)

# Write to a solution directional file, and print directional to console
bigMaze_obj.write_solution('Maze/solutions/bigMaze-solution.lay', False)



Goal found: (35, 1)
Execution time: 5.50103188 ms
Number of vertices: 1369
Number of edges: 2588
Number of steps taken: 470
Shortest path length found by DFS: 211


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓↕↔↔↔↔↔↔▓ ▓↕▓  ←←←←←←↑  ▓↕↔↔▓↔↔↔↔↕▓↕▓
▓↕▓▓▓▓▓▓▓ ▓↕▓▓▓↓▓↕▓▓▓↑▓▓▓↕▓▓▓▓▓▓▓↕▓↕▓
▓↔↔↔↔↕↔↔▓  ←←←←↓▓↕▓  ↑  ▓↕↔↔↔↔▓↕▓↕↔↔▓
▓▓▓▓▓↕▓▓▓▓▓↓▓▓▓↕▓↕▓ ▓↑▓▓▓↕▓▓▓▓▓↕▓↕▓▓▓
▓↕↔↔▓↕▓↕▓↕▓↓  ▓↕▓↕▓ ▓↑  ▓↕▓↕↔↔▓↕▓↔↔↕▓
▓↕▓▓▓↕▓↕▓↕▓↓▓▓▓↕▓▓▓▓▓↑▓▓▓↕▓↕▓▓▓↕▓▓▓↕▓
▓↔↔↕↔↔↔↔▓↔↔↓→→▓↕↔↔▓  ↑▓↔↔↕↔↔▓↕▓↕▓←←↑▓
▓▓▓↕▓▓▓▓▓▓▓▓▓↓▓▓▓▓▓▓▓↑▓▓▓↕▓▓▓↕▓↕▓↓▓↑▓
▓  ←←←←←←←←←←↓▓↑→→→→→→▓↕▓↕↔↔▓←←←←↓▓↑▓
▓ ▓↓▓▓▓▓▓↕▓↕▓▓▓↑▓↕▓↕▓▓▓↕▓↕▓▓▓↓▓▓▓↕▓↑▓
▓ ▓↓▓↔↔↔↔↕▓↕▓ ▓↑▓↕▓↕↔↔↔↔▓↕↔↔▓↓▓ ▓↕▓↑▓
▓ ▓↓▓↕▓▓▓▓▓▓▓ ▓↑▓▓▓▓▓▓▓▓▓↕▓▓▓↓▓ ▓▓▓↑▓
▓ ▓↓▓↕▓     ▓  ↑▓↔↔↔↔↕▓↔↔↕↔↔▓↓  ▓  ↑▓
▓▓▓↓▓▓▓ ▓ ▓▓▓▓▓↑▓▓▓▓▓↕▓▓▓↕▓▓▓↓▓▓▓▓▓↑▓
▓  ↓  ▓ ▓ ▓↔↔↑→→▓↕▓↔↔↕↔↔▓↕▓←←↓▓↕▓↕▓↑▓
▓ ▓↓▓ ▓ ▓ ▓▓▓↑▓▓▓↕▓▓▓↕▓▓▓↕▓↓▓↕▓↕▓↕▓↑▓
▓ ▓↓▓ ▓ ▓    ←←←←←←←←←←←←↑▓↓▓↕▓↑→→→→▓
▓▓▓↓▓▓▓▓▓▓▓ ▓ ▓↕▓▓▓▓▓↕▓▓▓↑▓↓▓▓▓↑▓▓▓▓▓
▓↔↔↓→→  ▓ ▓ ▓ ▓↕↔↔↔↔▓↕↔↔▓←←↓  ▓↑▓   ▓
▓▓▓▓▓↓▓ ▓ ▓▓▓▓▓▓▓▓▓↕▓▓▓▓▓▓▓▓▓▓▓↑▓ ▓▓▓
▓   ▓↓▓           ▓↕▓↕↔↔↔↔▓↑→→▓↑▓   ▓

In [12]:
openMaze = read_maze_from_file('Maze/openMaze.lay')

# Initialize small maze object and perform the DFS
openMaze_obj = Maze(openMaze)

openMaze_obj.solve()
openMaze_obj.calculate_complexity()
# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
openMaze_obj.print_path(False)

# Write to a solution directional file, and print directional to console
openMaze_obj.write_solution('Maze/solutions/openMaze-solution.lay', False)



Goal found: (21, 1)
Execution time: 16.50214195 ms
Number of vertices: 851
Number of edges: 2736
Number of steps taken: 704
Shortest path length found by DFS: 391


▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓↔↔↔↔↔↔↔↔↔↔↔↔↔↕←↑←↑←↑←↑←↑←↑←↑←↑←↑←↑S▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓←↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓▓
▓↔↔↔↔↔↔↔↔↔↔↔↕▓↕↓↑↓↑↓▓↕←↓←↓←↓←↓←↓←↓←↓▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓↕↓↑↓↑↓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓←↑←↑←↑←↑←↑←↑▓↕↓↑↓↑↓                ▓
▓↓↑↓↑↓↑↓↑↓↑↓↑▓↕↓↑↓↑↓                ▓
▓↓↑↓↑↓↑↓↑↓↑↓↑▓↕↓↑↓↑↓                ▓
▓↓↑↓↑↓↑↓↑↓↑↓↑←↑↓↑↓↑↓                ▓
▓↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓                ▓
▓↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓                ▓
▓G←↓←↓←↓←↓←↓←↓←↓←↓←↓                ▓

# Part 2: Search With Multiple Goals

## Key Differences between DFS in Part 1 and DFS in Part 2

- Goal Handling: In MazeMG, the goals are stored in a list and each goal is processed as it's found during the depth-first search (DFS). The algorithm continues even after a goal is found, until all goals have been found. In contrast, Maze only has one goal, so the DFS stops as soon as the goal is found.

- Goal Counter: MazeMG uses a goal_counter to keep track of the order in which the goals are found. This is not necessary in Maze as there is only one goal.

- Goal Removal: In MazeMG, once a goal is found, it's removed from the list of goals. This is not required in Maze as there is only one goal.

- DFS Return Value: In Maze, the DepthFirstSearch function returns True as soon as the goal is found, which immediately stops the DFS. In MazeMG, the DepthFirstSearch function doesn't return a value. Instead, it continues the DFS until all goals have been found.

- Backtracking: In both classes, if a position doesn't lead to a goal, it's removed from the path (this is called backtracking) and added to the backtracked list. However, in MazeMG, a position is only backtracked if it's not a goal, whereas in Maze, a position is backtracked if it doesn't lead to the goal.

In [23]:
class MazeMG:
    def __init__(self, maze):
        # Initialize the Maze object with the given maze
        self.maze = maze
        self.start = None
        self.goals = []  # List of goals
        self.original_goals = []
        self.free_spaces = []
        self.walls = []
        self.visited = []
        self.path = []
        self.backtracked = []
        self.directions = {}
        self.goal_counter = 1  # Initialize goal counter
        
        # Store the maze data in a 2d-array
        for i in range(len(maze)):
            for j in range(len(maze[i])):
                if maze[i][j] == 'P':
                    self.start = (i, j)
                elif maze[i][j] == '.':
                    self.goals.append((i, j))  # Add goal to list of goals
                    self.original_goals.append((i,j)) # Static goals list to reference goals later
                elif maze[i][j] == ' ':
                    self.free_spaces.append((i, j))
                elif maze[i][j] == '%':
                    self.walls.append((i, j))

    
    # Get the neighboring positions of a given position
    def get_neighbors(self, position):
        i, j = position
        neighbors = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
        return [pos for pos in neighbors if self.maze[pos[0]][pos[1]] in ['P', '.', ' '] and pos not in self.visited]

    # Used for directional result output
    def get_direction(self, pos1, pos2):
        # Get the direction from pos1 to pos2
        if pos2[0] > pos1[0]:
            return "↓"
        elif pos2[0] < pos1[0]:
            return "↑"
        elif pos2[1] > pos1[1]:
            return "→"
        elif pos2[1] < pos1[1]:
            return "←"
        else:
            # If the positions are the same, return the direction of the last movement
            return self.path[-1][1] if len(self.path) > 1 else None

    # =====================================================
    # Depth First Search (recursive)
    # 
    # Helper functions: get_direction, get_neighbors
    #
    # Author: Jacob Thieret
    #
    # Description: Recursively visits each neighboring position of the current position 
    #              until the goal is found or all possible paths have been exhausted. 
    #              Backtracking is performed by removing the current position from the path if 
    #              none of its neighbors lead to the goal. The process repeats until the goal is
    #              found or all positions have been visited.
    #
    # ====================================================
    def DepthFirstSearch(self, position=None):
        if position is None:
            position = self.start
        # Adds the position to the visited list.
        self.visited.append(position)
        
        # Determines the direction of movement from the previous position to the current one and adds it to the directions dictionary.
        if self.path:
            direction = self.get_direction(self.path[-1], position)
            self.directions[position] = direction
        
        # Adds the current position to the path.
        self.path.append(position)
        
        # Checks if the current position is a goal. If yes, it updates the maze, outputs a success message, and increments the goal counter.
        if position in self.goals:
            self.maze[position[0]][position[1]] = str(self.goal_counter)  # Replace goal with number of the order it was found in
            print(f"Goal {self.goal_counter} found: {position}")
            self.goal_counter += 1  
            self.goals.remove(position)  # Remove the goal from the list of goals
        
        # It gets the list of unvisited neighboring positions and recursively calls DepthFirstSearch on each one.
        neighbors = self.get_neighbors(position)
        for neighbor in neighbors:
            self.DepthFirstSearch(neighbor)
            
        # If none of the neighbors lead to a goal (i.e., DepthFirstSearch returns False for all of them), 
        # it removes the current position from the path (backtracking) and adds it to the backtracked list.
        if position not in self.goals:  # Only backtrack if the current position is not a goal
            self.path.pop()
            self.backtracked.append(position)
    
    # execute dfs search, measures the time it takes to run
    def solve(self):
        start_time = time.time()
        self.DepthFirstSearch()
        end_time = time.time()
        execution_time = (end_time - start_time) * 1000
        print(f"Execution time: {round(execution_time, 8)} ms")
    
    # Rough calculation of the complexity of the maze by counting the number of vertices and edges
    def calculate_complexity(self):
        num_vertices = len(self.maze) * len(self.maze[0])  # total number of cells
        num_edges = sum(sum(1 for cell in row if cell != '%') for row in self.maze) * 4  # total number of connections
        print(f"Number of vertices: {num_vertices}")
        print(f"Number of edges: {num_edges}")
    # ===================================
    # Output Functions
    # ===================================
        
    # Directional Output [file]
    # Write the solved maze with directional arrows and bidirectional symbols to a file
    def write_solutionDotGoal(self, filename,  toFile):
        print("\nOutput with goals as 'G'\n")
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
       
       # replace the numbered goals with their dots for better output formatting
        for goal in self.original_goals:
            solution_maze[goal[0]][goal[1]] = '.'
       
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        if toFile is True:
            directory = os.path.dirname(filename)
            if directory:
                os.makedirs(directory, exist_ok=True)
            with open(filename, 'w', encoding='utf-8') as file:
                for row in solution_maze:
                    row = ['S' if cell == 'P' else cell for cell in row]
                    row = ['G' if cell == '.' else cell for cell in row]
                    row = ['▓' if cell == '%' else cell for cell in row]
                    file.write(''.join(row) + '\n')
        else:
            for row in solution_maze:
                row = ['S' if cell == 'P' else cell for cell in row]
                row = ['G' if cell == '.' else cell for cell in row]
                row = ['▓' if cell == '%' else cell for cell in row]
                print(''.join(row))
    
    # Directional Output Numbered Goals [console]
    # Write the solved maze with directional arrows and bidirectional symbols to the console
    def write_solutionToConsoleNumGoal(self):
        print("\nOutput with goals as numbers in order found\n")
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        for row in solution_maze:
            row = ['S' if cell == 'P' else cell for cell in row]
            row = ['▓' if cell == '%' else cell for cell in row]
            print(''.join(row))

    # Print the number of steps taken and the path taken
    def print_stepCount(self):
        # Number of steps taken
        print(f"Number of steps taken: {len(self.visited)}")    

In [24]:
import time

tinySearch = read_maze_from_file('Maze/tinySearch.lay')

# Initialize tiny search maze object and perform the DFS
tinySearch_obj = MazeMG(tinySearch)

tinySearch_obj.solve()
tinySearch_obj.calculate_complexity()
# Print the step count
tinySearch_obj.print_stepCount()

# Print solution with goals as nums to console
tinySearch_obj.write_solutionToConsoleNumGoal()
# Print solution to file if true and to console if false
tinySearch_obj.write_solutionDotGoal('Maze/solutions/tinySearch-solution.lay', False)



Goal 1 found: (2, 4)
Goal 2 found: (1, 2)
Goal 3 found: (1, 1)
Goal 4 found: (1, 6)
Goal 5 found: (1, 7)
Goal 6 found: (4, 7)
Goal 7 found: (5, 7)
Goal 8 found: (5, 3)
Goal 9 found: (4, 1)
Goal 10 found: (5, 1)
Execution time: 0.0 ms
Number of vertices: 63
Number of edges: 100
Number of steps taken: 27

Output with goals as numbers in order found

▓▓▓▓▓▓▓▓▓
▓32↔↕↔45▓
▓▓▓▓1▓▓↕▓
▓↔↔↔S↔↔↕▓
▓9▓▓↕▓▓6▓
▓10▓8↔↔↔7▓
▓▓▓▓▓▓▓▓▓

Output with goals as 'G'

▓▓▓▓▓▓▓▓▓
▓GG↔↕↔GG▓
▓▓▓▓G▓▓↕▓
▓↔↔↔S↔↔↕▓
▓G▓▓↕▓▓G▓
▓G▓G↔↔↔G▓
▓▓▓▓▓▓▓▓▓


In [25]:
smallSearch = read_maze_from_file('Maze/smallSearch.lay')

# Initialize small search maze object and perform the DFS
smallSearch_obj = MazeMG(smallSearch)

smallSearch_obj.solve()
smallSearch_obj.calculate_complexity()
# Print the step count
smallSearch_obj.print_stepCount()

# Print solution with goals as nums to console
smallSearch_obj.write_solutionToConsoleNumGoal()
# Print solution to file if true and to console if false
smallSearch_obj.write_solutionDotGoal('Maze/solutions/smallSearch-solution.lay', False)

Goal 1 found: (2, 13)
Goal 2 found: (1, 13)
Goal 3 found: (2, 10)
Goal 4 found: (3, 10)
Goal 5 found: (3, 9)
Goal 6 found: (3, 8)
Goal 7 found: (3, 7)
Goal 8 found: (2, 7)
Goal 9 found: (2, 4)
Goal 10 found: (1, 1)
Goal 11 found: (2, 1)
Goal 12 found: (3, 6)
Goal 13 found: (1, 14)
Goal 14 found: (1, 15)
Goal 15 found: (1, 18)
Goal 16 found: (2, 18)
Goal 17 found: (3, 18)
Execution time: 0.5004406 ms
Number of vertices: 100
Number of edges: 156
Number of steps taken: 42

Output with goals as numbers in order found

▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓10↔↔↔↔↔↕↔↔↔↔↔21314S↔15▓
▓11▓▓9▓▓8▓▓3▓▓1▓▓↕▓16▓
▓↕▓▓↕▓127654↔↔↔↔↔↕▓17▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

Output with goals as 'G'

▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓G↔↔↔↔↔↕↔↔↔↔↔GGGS↔G▓
▓G▓▓G▓▓G▓▓G▓▓G▓▓↕▓G▓
▓↕▓▓↕▓GGGGG↔↔↔↔↔↕▓G▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓


In [26]:
trickySearch = read_maze_from_file('Maze/trickySearch.lay')

# Initialize small search maze object and perform the DFS
trickySearch_obj = MazeMG(trickySearch)

trickySearch_obj.solve()
trickySearch_obj.calculate_complexity()
# Print the step count
trickySearch_obj.print_stepCount()

# Print solution with goals as nums to console
trickySearch_obj.write_solutionToConsoleNumGoal()
# Print solution to file if true and to console if false
trickySearch_obj.write_solutionDotGoal('Maze/solutions/trickySearch_obj-solution.lay', False)

Goal 1 found: (2, 7)
Goal 2 found: (2, 4)
Goal 3 found: (2, 1)
Goal 4 found: (1, 1)
Goal 5 found: (2, 10)
Goal 6 found: (2, 13)
Goal 7 found: (1, 13)
Goal 8 found: (1, 14)
Goal 9 found: (5, 5)
Goal 10 found: (5, 4)
Goal 11 found: (5, 3)
Goal 12 found: (5, 2)
Goal 13 found: (5, 1)
Execution time: 0.49972534 ms
Number of vertices: 140
Number of edges: 240
Number of steps taken: 64

Output with goals as numbers in order found

▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓4↔↔↔↔↔↕↔↔↔↔↔78▓↕↔↔▓
▓3▓▓2▓▓1▓▓5▓▓6▓▓↕▓↕▓
▓↔↔↔↕↔↔↔↔S↔↔↔↔↔↔↔▓↕▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓↕▓
▓131211109↔↔↔↔↔↔↔↔↔↔↔↔↕▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

Output with goals as 'G'

▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
▓G↔↔↔↔↔↕↔↔↔↔↔GG▓↕↔↔▓
▓G▓▓G▓▓G▓▓G▓▓G▓▓↕▓↕▓
▓↔↔↔↕↔↔↔↔S↔↔↔↔↔↔↔▓↕▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓↕▓
▓GGGGG↔↔↔↔↔↔↔↔↔↔↔↔↕▓
▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
